# Modifications to the standard Gordon equilibrium solver

The Gordon equilibrium solver works well for reactions in solution or gases. Our 2 key modifications are to 

1) Modify the chemical potential equation to be zero if we run out of a solid/liquid substance (n_i = 0) and the chemical potential difference is positive ($\mu_i - \sum A \vec{\lambda} > 0$) - we have used the substance up, and there is no way to satisfy the chemical potential equation. Note that if the chemical potential difference is negative, we should instead create more of the substance.

2) Not sure if this is different than their approach, but use the exact chemical potential of each element at each iteration.

Then hopefully the same iterative procedure should work well?

In [227]:
%load_ext autoreload
%autoreload 2
import pchem.calorimetry as cal
import numpy as np
import sympy as sm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test case showing good performance for normal chemical equilibrium

In [228]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
P  = cal.Substance2("P(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(P=1))
D  = cal.Substance2("D(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1))
PD = cal.Substance2("PD(aq)", state="aq", H0=0, S0=50.0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1, P=1))

chemicals = {0: S, 1: P, 2: D, 3: PD}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:50.0, 1: 1.0, 2: 1.0, 3: 0.0})

In [229]:
K = np.exp(50.0/8.3145)

In [230]:
x = sm.symbols('x')

x_solved = float(sm.solve(K - x/ (1-x)**2, x)[0])

In [231]:
out, iterations = s0._solve_iterative()

[ 1.  1.  0. 50.]


In [232]:
np.allclose( np.array([1-x_solved, 1-x_solved, x_solved]), out[:3])

True

## Normal chemical equilibrium: temperature change

In [133]:
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")

T=298.15


In [134]:
out, iterations = s0._solve_iterative_dH(dH=100.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")

cP = 2500.0
dT = 100/2500.0

np.allclose(T, s0.T + dT) # Good!

T=298.189999941779


True

## Exothermic reaction

Here the temperature changes will be coupled to the chemical reaction.

In [135]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
P  = cal.Substance2("P(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(P=1))
D  = cal.Substance2("D(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1))
PD = cal.Substance2("PD(aq)", state="aq", H0=-50*298.15, S0=0.0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1, P=1))

chemicals = {0: S, 1: P, 2: D, 3: PD}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:50.0, 1: 1.0, 2: 1.0, 3: 0.0})

In [136]:
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.82532262956835
n_moles=array([4.82423609e-02, 4.82423609e-02, 9.51757639e-01, 5.00000722e+01])


In [137]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.80913594376807
n_moles=array([ 0.05095664,  0.05095664,  0.94904336, 50.00008497])


In [138]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.809181706801
n_moles=array([ 0.05094884,  0.05094884,  0.94905116, 50.00009162])


In [139]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.80918106502133
n_moles=array([ 0.05094886,  0.05094886,  0.94905114, 50.00009613])


In [140]:
iterations

[4.3162057661305653e-07,
 5.280708847976712e-08,
 5.261091473585111e-08,
 5.241617628826134e-08,
 5.222286603157045e-08,
 5.2030955544069e-08,
 5.1840437720329646e-08,
 5.1651298349497665e-08,
 5.1463516115290986e-08,
 5.127708391228225e-08,
 5.109198042418939e-08,
 5.090819854558504e-08,
 5.072571696018713e-08,
 5.0544521457140945e-08,
 5.036460493101913e-08,
 5.018595317096697e-08,
 5.0008551966129744e-08,
 4.983238710565274e-08,
 4.965744437868125e-08,
 4.9483716679787904e-08,
 4.9311182692690636e-08,
 4.9139842417389445e-08,
 4.89696674321749e-08,
 4.880066484247436e-08,
 4.8632806226578396e-08,
 4.8466084479059646e-08,
 4.830049959991811e-08,
 4.8136023167444364e-08,
 4.79726551816384e-08,
 4.781037432621815e-08,
 4.7649180601183616e-08,
 4.748905979568008e-08,
 4.733000480428018e-08,
 4.717199431070185e-08,
 4.7015028314945084e-08,
 4.685909260615517e-08,
 4.67041729734774e-08,
 4.655026941691176e-08,
 4.639736062017619e-08,
 4.624544658327068e-08,
 4.609451309534052e-08,
 4.5944

## Phase transition

In [325]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
Ssolid = cal.Substance2("S(s)", state="s", H0=-10000, S0=-20.0, cP=40.0, molarMass=100.0, density=1.05, atoms=dict(S=1))
chemicals = {0: Ssolid, 1: S}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:10.0, 1: 0.0})

In [326]:
x0_pt = np.r_[s0.x, 4037/(s0.T*8.3145), s0.T]

A, y = s0._fHi(x0_pt, 400.0)

In [327]:
s0.chem_labels

('S(l)', 'S(s)')

In [328]:
y

array([-0.        ,  1.62850013,  0.        ,  0.16135746])

In [329]:
lb = np.array([0, -10, -np.inf, -np.inf])

In [330]:
dy = A @ np.array([0,0,x0_pt[2],0])

In [331]:
m=np.array([True, True, False, True])

In [332]:
y3 = (y+dy)

In [333]:
A3 = A[:, m]

In [334]:
y3

array([-1.62850013,  0.        ,  0.        ,  0.16135746])

In [335]:
lb = np.array([0, -10, -np.inf])
y_out = optimize.lsq_linear(A3, y3, bounds=(lb, np.inf))

In [336]:
x_new = np.r_[x0_pt[:2] + y_out.x[:2], 0.0, x0_pt[-1]*np.exp(y_out.x[-1])]

In [337]:
mu2 = s0.mu(x=x_new[:2], T=x_new[-1])/(8.3145*x_new[-1])

In [338]:
mu2

array([-1.62198613])

In [344]:
A, y = s0._fHi(x_new, 2000.0)
dy = A @ np.array([0,0,mu2[0],0])
y3 = y - dy
A3 = A[:, m]

In [345]:
A

array([[ 0.        ,  0.        , -1.        , -0.        ],
       [ 0.        ,  0.        , -1.        ,  4.03393642],
       [ 1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -4.03393642,  0.        , 48.15683445]])

In [352]:
U, s, Vh = linalg.svd(A)

$$A = U \sigma V^T$$

$$U^T U \sigma V^T x = y$$

$$ x = V \sigma^{-1} U^T y$$

In [364]:
Vh.T @ np.diag(s**-1) @ U.T @ y

array([-4.65934073e+00,  4.65934073e+00, -1.35185608e-16,  4.03700000e-01])

In [365]:
optimize.nonlin.l

array([-4.65934073,  4.65934073, -0.        ,  0.4037    ])

In [340]:
lb = np.r_[-x_new[:2], -np.inf]
y_out2 = optimize.lsq_linear(A3, y3, bounds=(lb, np.inf))

In [341]:
x_new2 = np.r_[x_new[:2] + y_out2.x[:2], 0.0, x_new[-1]*np.exp(y_out2.x[-1])]

In [343]:
# optimize.least_squares()

array([1.80722637e-01, 9.81927736e+00, 0.00000000e+00, 2.98631842e+02])

In [237]:
Aw = w @ A
yw = w @ y

In [238]:
lsq_lin = optimize.lsq_linear(Aw, yw, bounds=(lb, np.inf))

In [231]:
A

array([ 0.        ,  0.        , -1.62850013,  0.        ])

In [233]:
v

array([ 0.        ,  0.        , -1.62850013,  0.        ])

In [240]:
x1 = np.r_[x0_pt[:2] + lsq_lin.x[:2], lsq_lin.x[2], x0_pt[-1]*np.exp(lsq_lin.x[-1])]

In [241]:
x1

array([ 2.46833384e-01,  9.75316662e+00, -1.71199084e+00,  2.92042586e+02])

In [228]:
A @ lsq_lin.x - y

array([ 1.71199084e+00, -2.22044605e-16,  0.00000000e+00,  1.11022302e-16])

In [108]:
from scipy import linalg, optimize

In [109]:
optimize.lsq_linear()

TypeError: lstsq() missing 2 required positional arguments: 'a' and 'b'

In [155]:
C = np.array([[1, 0, 0], [0, 1, 0], [0, 0,0], [0, 0, 1]]).T

In [157]:
C

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1]])

In [143]:
y_r = C.T @ y

In [147]:
C.T @ A @ C

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  4.03393642],
       [ 0.        , -4.03393642, 48.10872572]])

In [144]:
C @ y_r

array([0.        , 1.62850013, 0.        , 0.16135746])

In [145]:
y_r

array([0.        , 1.62850013, 0.16135746])

In [148]:
condensed_phase_active = [True, False]

In [161]:
def make_C(s, active_phases, include_T=False):
    N_condensed_active = sum(active_phases)
    N_condensed = s.N_chem - s.N_aq
    N_total = s.N_chem + s.N_atoms
    if include_T:
        N_total += 1
    
    N_active = N_total + (N_condensed_active - N_condensed) # Remove inactive condensed species...
    C = np.zeros((N_active, N_total))
    
    for i in range(s.N_aq):
        C[i, i] = 1
    
    i_curr = s.N_aq
    for i in range(s.N_aq, s.N_chem):
        j = i - s.N_aq
        if active_phases[j]:
            C[i_curr, i] = 1
            i_curr += 1
    
    for i in range(s.N_chem, N_total):
        C[i_curr, i] = 1
        i_curr+= 1
    
    return C
    

In [181]:
C_active = make_C(s0, [False, True], True)

In [182]:
A, y = s0._fHi(x0_pt, 400.0)

In [196]:
A_active = C @ A @C_active.T
y_active = C @ y

In [184]:
from scipy import linalg

In [203]:
m = np.array([False, True, True, True], dtype=bool)

In [209]:
A_active = A[m][:, m] # Choose active columns...
y_active = y[m]

In [208]:
linalg.solve(A_active, y_active)

array([-0.        , -1.61497024,  0.00335402])

array([[ 0.        ,  0.        , -1.        , -0.        ],
       [ 0.        ,  0.        , -1.        ,  4.03393642],
       [ 1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -4.03393642,  0.        , 48.10872572]])